# 第三次作業

## Basline
將第一次作業的 QA 檢索聊天機器人，重新透過 LangChain 進行實作

In [4]:
import torch
from transformers import BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

# 1. Document Loading

In [9]:
from langchain.document_loaders import TextLoader

In [11]:
fiel_dir = "/content/火影忍者.txt"

loader = TextLoader(fiel_dir)
pages = loader.load()

# 2. Splitting

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=30)
texts_chunks = text_splitter.split_documents(pages)

# 3. Storage

## 3.1 Embedding

In [16]:
from langchain.embeddings import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
# 初始化 Embedding 模型
embedding_func = HuggingFaceEmbeddings(
    model_name="infgrad/stella-base-zh-v3-1792d",
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True})

# 將字句轉換為向量
# a = embedding_func.embed_query('突襲式發表！蘋果推 2 款 M3 MacBook Air，強調 AI 、遊戲效能皆強化')
# b = embedding_func.embed_query('蘋果最新M3版MacBook Air突襲登場！6亮點下放1技術不漲價 M2版還降3000元')

# 計算相似度
# cosine_similarity([a], [b])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


modules.json:   0%|          | 0.00/311 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/32.1k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Some weights of BertModel were not initialized from the model checkpoint at infgrad/stella-base-zh-v3-1792d and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/5.51M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

## 3.2 Vectore Store

In [18]:
from langchain.vectorstores import Chroma

In [21]:
# load it into Chroma
db = Chroma.from_documents(texts_chunks, embedding_func)

# query it
query = "螺旋手裡劍很厲害嗎？主要是誰的招式？"
docs = db.similarity_search_with_score(query)

# 4. Retrival

In [5]:
MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"


# 量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True)

# llm 初始化
llm = HuggingFacePipeline.from_model_id(
    model_id=MODEL_NAME,
    task="text-generation",
    model_kwargs=dict(
        torch_dtype=torch.float16,
        trust_remote_code=True,
        device_map="auto",
        quantization_config=quantization_config),
    pipeline_kwargs=dict(
        max_new_tokens=1024,
        temperature=0.0001,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.15) )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.79M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [6]:
# Prompt 模板
template = """
<s>
請你做為一個大型語言模型的專家，並根據下方所提供的資訊，來回答使用者的提問。

[INST]
{context}

{question}
[/INST]
"""
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [23]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

result = qa_chain({"query": query})

# LLM 提供的回答
print("回覆：")
print(result["result"])

# 參考的相關資訊
print("參考資訊：")
print(result["source_documents"])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


回覆：

<s>
請你做為一個大型語言模型的專家，並根據下方所提供的資訊，來回答使用者的提問。

[INST]
說明:鳴人將螺旋丸加入性質變化之後開發出來的忍術，是鳴人在仙人模式或尾獸模式時才能使出的禁術，利用仙人模式或尾獸模式提高查克拉密度，將螺旋手裡劍拋出攻擊，否則會使施術者手臂嚴重創傷，甚至永遠無法凝聚查克拉。命中後查克拉會化成無數細小的風刃可把對手體內所有的經脈系統及細胞擊潰，攻擊次數就連卡卡西的寫輪眼也無法看透。鳴人在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。
結印:無

結印:無
主要使用者:漩渦鳴人與第四代火影"
"招式:風遁‧迷你螺旋手裡劍
說明:迷你的螺旋手裡劍，由尾獸狀態的鳴人使出，可切割任何東西。
結印:無
主要使用者:漩渦鳴人"
"招式:熔遁‧螺旋手裡劍
說明:在螺旋手裡劍中加入四尾孫悟空的血繼限界熔遁查克拉，形成帶有熔岩的螺旋手裡劍。
結印:無
主要使用者:漩渦鳴人"
"招式:惑星螺旋丸

螺旋手裡劍很厲害嗎？主要是誰的招式？
[/INST]
螺旋手裡劍是漩渦鳴人的招式，屬於他的忍術「風�・迷你螺旋手裡劍」和「熔�・螺旋手裡劍」。這兩項忍者術都具有強大的破壞力，可以切斷一切物體、甚至穿透防禦措施。
參考資訊：
[Document(metadata={'source': '/content/火影忍者.txt'}, page_content='說明:鳴人將螺旋丸加入性質變化之後開發出來的忍術，是鳴人在仙人模式或尾獸模式時才能使出的禁術，利用仙人模式或尾獸模式提高查克拉密度，將螺旋手裡劍拋出攻擊，否則會使施術者手臂嚴重創傷，甚至永遠無法凝聚查克拉。命中後查克拉會化成無數細小的風刃可把對手體內所有的經脈系統及細胞擊潰，攻擊次數就連卡卡西的寫輪眼也無法看透。鳴人在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。\n結印:無'), Document(metadata={'source': '/content/火影忍者.txt'}, page_content='結印:無\n主要使用者:漩渦鳴人與第四代火影"\n"招式:風遁‧迷你螺旋手裡劍\n說明:迷你的螺旋手裡劍，由尾獸狀態的鳴人使出，可切割任何東西。\n結印:無\n主要使用者:漩渦鳴人"\n"招式:熔遁‧螺旋手裡劍\n說明:在螺旋手裡劍中加入四尾孫悟空的血繼限界熔遁查

## Advance
延伸 Basline 的內容，測試看看不同的 chunk 大小，對於生成結果的影響

In [37]:
import numpy as np

In [44]:
for chunk_size in np.append(50, np.arange(100, 1001, 100)):
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=30)
  texts_chunks = text_splitter.split_documents(pages)

  # # 初始化 Embedding 模型
  # embedding_func = HuggingFaceEmbeddings(
  #   model_name="infgrad/stella-base-zh-v3-1792d",
  #   model_kwargs={"device": "cuda"},
  #   encode_kwargs={"normalize_embeddings": True})

  # load it into Chroma
  db = Chroma.from_documents(texts_chunks, embedding_func)

  # query it
  query = "螺旋手裡劍很厲害嗎？主要是誰的招式？"
  docs = db.similarity_search_with_score(query)

  # MODEL_NAME = "MediaTek-Research/Breeze-7B-Instruct-v0_1"


  # # 量化參數
  # quantization_config = BitsAndBytesConfig(
  #     load_in_4bit=True,
  #     bnb_4bit_compute_dtype=torch.float16,
  #     bnb_4bit_quant_type="nf4",
  #     bnb_4bit_use_double_quant=True)

  # # llm 初始化
  # llm = HuggingFacePipeline.from_model_id(
  #     model_id=MODEL_NAME,
  #     task="text-generation",
  #     model_kwargs=dict(
  #         torch_dtype=torch.float16,
  #         trust_remote_code=True,
  #         device_map="auto",
  #         quantization_config=quantization_config),
  #     pipeline_kwargs=dict(
  #         max_new_tokens=1024,
  #         temperature=0.0001,
  #         top_p=0.95,
  #         do_sample=True,
  #         repetition_penalty=1.15) )

  # Prompt 模板
  template = """
  <s>
  請你做為一個大型語言模型的專家，並根據下方所提供的資訊，來回答使用者的提問。

  [INST]
  {context}

  {question}
  [/INST]
  """
  prompt = PromptTemplate(template=template, input_variables=["context", "question"])

  qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 2}),
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt})

  result = qa_chain({"query": query})

  print(f'****************chunk_size: {chunk_size} ****************')
  # LLM 提供的回答
  print("回覆：")
  print(result["result"])

  # 參考的相關資訊
  print("參考資訊：")
  print(result["source_documents"])
  print('==='*80)

****************chunk_size: 50 ****************
回覆：

  <s>
  請你做為一個大型語言模型的專家，並根據下方所提供的資訊，來回答使用者的提問。

  [INST]
  在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。

在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。

  螺旋手裡劍很厲害嗎？主要是誰的招式？
  [/INST]
  螺旋手裡劍（Rasengan）是鳴人的招牌忍者技術，也是他最著名的忍術之一。它被認為是鳴人的最強忍術，因為它是利用風屬性的能量和自身的速度與力量，將空氣壓縮成高密度的旋風，形成一柄旋轉式的劍形攻擊。螺旋手裡劍非常有效地結合了風屬性、速度、力量以及空間控制能力，使之成為鳴人的重要武器。
參考資訊：
[Document(metadata={'source': '/content/火影忍者.txt'}, page_content='在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。'), Document(metadata={'source': '/content/火影忍者.txt'}, page_content='在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。')]
****************chunk_size: 100 ****************
回覆：

  <s>
  請你做為一個大型語言模型的專家，並根據下方所提供的資訊，來回答使用者的提問。

  [INST]
  在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。

在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。

  螺旋手裡劍很厲害嗎？主要是誰的招式？
  [/INST]
  螺旋手裡劍（Rasengan）是鳴人的招牌忍者技巧，被認為是他的最強忍術之一。它具有高攻擊力和速度，可以有效地對付敵人。螺旋手裡劍是由旋風輪（Ryūton）與手裏劍（Kenkaku-dan）這兩種忍術結合而成，因此也稱為「風遁・螺旋手裏劍」。這個技能的威力取決於忍者的實力，所以在鳴人的手中，它無疑是非常強大且致命的招式。
參考資訊：
[Document(metadata={'source': '/content/火影忍者.txt'}, p

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


****************chunk_size: 700 ****************
回覆：

  <s>
  請你做為一個大型語言模型的專家，並根據下方所提供的資訊，來回答使用者的提問。

  [INST]
  在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。

在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。

  螺旋手裡劍很厲害嗎？主要是誰的招式？
  [/INST]
  螺旋手裡劍（Rasengan）是鳴人的招牌忍者技巧，被認為是他的最強忍術之一。它具有高攻擊力和速度，可以有效地對付敵人。螺旋手裡劍是由旋風輪（Ryūton）與手裏劍（Kenkaku-dan）這兩種忍術結合而成，因此也稱為「風遁・螺旋手裏劍」。這個技能的威力取決於忍者的實力，所以在鳴人的手中，它無疑是非常強大且致命的招式。
參考資訊：
[Document(metadata={'source': '/content/火影忍者.txt'}, page_content='在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。'), Document(metadata={'source': '/content/火影忍者.txt'}, page_content='在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。')]
****************chunk_size: 800 ****************
回覆：

  <s>
  請你做為一個大型語言模型的專家，並根據下方所提供的資訊，來回答使用者的提問。

  [INST]
  在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。

在培因之戰首次使用完全完成的風遁·螺旋手裏劍，是鳴人最強的忍術。

  螺旋手裡劍很厲害嗎？主要是誰的招式？
  [/INST]
  螺旋手裡劍（Rasengan）是鳴人的招牌忍者技術，也是他最著名的忍術之一。它被認為是鳴人的最強忍術，因為它是利用風屬性的能量和自身的速度與力量，將空氣壓縮成高密度的旋風，形成一柄旋轉式的劍形攻擊。螺旋手裡劍非常有效地結合了風屬性、速度、力量以及空間控制能力，使之成為鳴人的重要武器。
參考資訊：
[Document(metadata={'source': '/content/火影忍者.txt'}, 